# IMPORT

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 
import numpy as np
import statistics
from sklearn import preprocessing,svm
from sklearn.metrics import *
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from itertools import product, combinations
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, train_test_split, LeaveOneOut, RepeatedStratifiedKFold
import itertools
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
import cvxopt
import matplotlib.mlab as mlab
import seaborn
import pickle

from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

# DATA LOADING

In [ ]:
df2_gb = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Generated Features/Global features/global_measures_binary_combined_0_2.csv'
df2_gw = '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Generated Features/Global features/global_combined_weighted.csv'

In [ ]:
df2 = pd.read_csv(df2_gb)
df2.drop(['Unnamed: 0',], axis = 1, inplace = True, errors='ignore')
index = df2[(df2['Subject'] == 2155)].index
df2.drop(index , inplace=True)
df2

,Subject,Schizophrenic,Approximation and Heuristics Node Connectivity,Approximation and Heuristics Max Independent Sets,Approximation and Heuristics Max Cliques,Approximation and Heuristics Clique Removal,Approximation and Heuristics Large Clique Size,Approximation and Heuristics Average Clustering,Approximation and Heuristics Diameter,Approximation and Heuristics Min Edge Dominating Set,...,Chordal,Maximal Cliques,Num of isolates,Non Randomness wrt Random Model,Transitivity,Connected Components,Diameter,1 Edge Connected,2 Edge Connected,3 Edge Connected
0,2001,0,10,"{160, 129, 2, 3, 0, 6, 7, 11, 108, 142, 46, 14...","{32, 33, 2, 38, 72, 73, 43, 140, 148, 149, 22,...","({160, 129, 2, 3, 0, 6, 7, 11, 108, 142, 46, 1...",17,0.465,3,"{(54, 55), (86, 87), (125, 126), (17, 20), (40...",...,False,1813,0,-0.570663,0.503559,1,3,True,True,True
1,2002,0,11,"{0, 129, 1, 8, 10, 12, 141, 143, 18, 152, 93, 94}","{12, 14, 15, 145, 146, 17, 158, 34, 35, 36, 68...","({0, 129, 1, 8, 10, 12, 141, 143, 18, 152, 93,...",25,0.563,3,"{(125, 126), (26, 28), (17, 20), (72, 73), (35...",...,False,5810,0,10.101452,0.548962,1,3,True,True,True
2,2003,0,1,"{96, 1, 0, 5, 8, 137, 11, 141, 46, 49, 87, 24,...","{32, 34, 68, 36, 70, 40, 42, 107, 13, 111, 16,...","({96, 1, 0, 5, 8, 137, 11, 141, 46, 49, 87, 24...",17,0.489,4,"{(54, 55), (86, 87), (125, 126), (40, 41), (72...",...,False,3860,0,7.200119,0.492561,1,4,True,False,False
3,2004,0,10,"{4, 39, 8, 7, 74, 13, 51, 148, 20, 19, 27, 61,...","{64, 102, 103, 104, 46, 50, 51, 52, 54, 58, 29}","({4, 39, 8, 7, 74, 13, 51, 148, 20, 19, 27, 61...",16,0.461,3,"{(134, 137), (65, 67), (54, 55), (30, 41), (15...",...,False,3505,0,7.945752,0.464850,1,3,True,True,True
4,2005,0,15,"{0, 129, 1, 99, 4, 6, 7, 41, 138, 108, 150, 15...","{128, 0, 131, 132, 133, 134, 136, 11, 83, 84, ...","({0, 129, 1, 99, 4, 6, 7, 41, 138, 108, 150, 1...",14,0.490,3,"{(54, 55), (86, 87), (156, 157), (40, 41), (72...",...,False,3339,0,5.800827,0.472348,1,3,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,4156,1,11,"{161, 65, 2, 132, 100, 58, 36, 11, 109, 78, 14...","{160, 33, 32, 136, 8, 138, 139, 140, 146, 147,...","({161, 65, 2, 132, 100, 58, 36, 11, 109, 78, 1...",20,0.500,3,"{(125, 126), (40, 41), (18, 19), (34, 35), (23...",...,False,4429,0,-4.078784,0.498773,1,3,True,True,True
239,4159,1,12,"{160, 1, 163, 100, 133, 36, 26, 10, 12, 19, 11...","{132, 5, 6, 7, 4, 9, 34, 35, 45, 46, 64, 65, 7...","({160, 1, 163, 100, 133, 36, 26, 10, 12, 19, 1...",24,0.531,3,"{(43, 48), (156, 157), (40, 41), (110, 111), (...",...,False,3767,0,8.362277,0.549560,1,3,True,True,True
240,4169,1,5,"{130, 132, 133, 100, 137, 74, 42, 11, 141, 143...","{64, 65, 4, 5, 6, 7, 9, 44, 45, 112, 81, 116, ...","({130, 132, 133, 100, 137, 74, 42, 11, 141, 14...",21,0.526,3,"{(86, 87), (33, 36), (72, 73), (141, 145), (18...",...,False,2133,0,7.042686,0.499686,1,3,True,True,True
241,4170,1,6,NaN,"{11, 12, 13, 14, 15, 144, 24, 27, 29, 32, 33, ...","({161, 1, 4, 7, 16, 49, 23, 58, 156, 31}, [{0,...",35,0.623,3,"{(62, 64), (125, 126), (156, 157), (18, 19), (...",...,False,102687,0,18.747141,0.664289,1,3,True,True,True


In [ ]:
df = pd.read_csv(df2_gw)
df.drop(['Unnamed: 0'], axis = 1, inplace = True, errors='ignore')
index = df[(df['Subject'] == 2155)].index
df.drop(index , inplace=True)
df

,Subject,Average shortest path,Stoer Wagner cuts,Wiener Index,Dijkstra path,Max weight matching,Conductance,Randomized Partitioning Heuristic
0,2001,0.008042,19.841134,107.483595,8,82,116.917205,"(1340.0942298403615, ({0, 3, 4, 5, 6, 12, 13, ..."
1,2002,0.012136,19.315941,162.212200,4,82,126.843205,"(1491.6328930370037, ({1, 129, 130, 4, 5, 6, 7..."
2,2003,0.010182,16.432399,136.090766,3,82,111.259694,"(1302.932983262392, ({1, 3, 5, 6, 7, 8, 12, 13..."
3,2004,0.009460,20.374916,126.446053,6,82,119.750835,"(1283.965973913516, ({0, 1, 3, 4, 6, 8, 11, 13..."
4,2005,0.011182,20.140523,149.462698,7,82,114.872774,"(1304.7065863045175, ({1, 2, 3, 4, 7, 8, 9, 10..."
...,...,...,...,...,...,...,...,...
261,4167,0.011655,19.283984,155.780433,6,82,NaN,"(1456.528461115162, ({1, 8, 9, 10, 12, 13, 15,..."
262,4168,0.012280,20.846043,164.140639,7,82,NaN,"(1459.219925223858, ({0, 1, 2, 4, 8, 10, 12, 1..."
263,4169,0.011931,18.323694,159.472844,8,82,NaN,"(1484.4315153492448, ({0, 1, 3, 4, 5, 8, 9, 12..."
264,4170,0.013469,20.179314,180.026202,4,82,NaN,"(1639.2320994076063, ({1, 2, 4, 5, 6, 7, 8, 9,..."


In [ ]:
df = df.merge(df2, on='Subject', how='inner',)

# DATA CLEANING

In [ ]:
df = df.drop(['Approximation and Heuristics Clique Removal','Approximation and Heuristics Min Edge Dominating Set', 'Approximation and Heuristics Maximal Matching',
         'Approximation and Heuristics Ramsay', 'Approximation and Heuristics treewidth Min Wt Vertex Cover','Conductance'], axis=1)
df

,Subject,Average shortest path,Stoer Wagner cuts,Wiener Index,Dijkstra path,Max weight matching,Randomized Partitioning Heuristic,Schizophrenic,Approximation and Heuristics Node Connectivity,Approximation and Heuristics Max Independent Sets,...,Chordal,Maximal Cliques,Num of isolates,Non Randomness wrt Random Model,Transitivity,Connected Components,Diameter,1 Edge Connected,2 Edge Connected,3 Edge Connected
0,2001,0.008042,19.841134,107.483595,8,82,"(1340.0942298403615, ({0, 3, 4, 5, 6, 12, 13, ...",0,10,"{160, 129, 2, 3, 0, 6, 7, 11, 108, 142, 46, 14...",...,False,1813,0,-0.570663,0.503559,1,3,True,True,True
1,2002,0.012136,19.315941,162.212200,4,82,"(1491.6328930370037, ({1, 129, 130, 4, 5, 6, 7...",0,11,"{0, 129, 1, 8, 10, 12, 141, 143, 18, 152, 93, 94}",...,False,5810,0,10.101452,0.548962,1,3,True,True,True
2,2003,0.010182,16.432399,136.090766,3,82,"(1302.932983262392, ({1, 3, 5, 6, 7, 8, 12, 13...",0,1,"{96, 1, 0, 5, 8, 137, 11, 141, 46, 49, 87, 24,...",...,False,3860,0,7.200119,0.492561,1,4,True,False,False
3,2004,0.009460,20.374916,126.446053,6,82,"(1283.965973913516, ({0, 1, 3, 4, 6, 8, 11, 13...",0,10,"{4, 39, 8, 7, 74, 13, 51, 148, 20, 19, 27, 61,...",...,False,3505,0,7.945752,0.464850,1,3,True,True,True
4,2005,0.011182,20.140523,149.462698,7,82,"(1304.7065863045175, ({1, 2, 3, 4, 7, 8, 9, 10...",0,15,"{0, 129, 1, 99, 4, 6, 7, 41, 138, 108, 150, 15...",...,False,3339,0,5.800827,0.472348,1,3,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,4156,0.010345,21.208409,138.275598,4,82,"(1426.8818395540534, ({0, 3, 4, 7, 9, 10, 12, ...",1,11,"{161, 65, 2, 132, 100, 58, 36, 11, 109, 78, 14...",...,False,4429,0,-4.078784,0.498773,1,3,True,True,True
239,4159,0.010946,21.128360,146.306992,9,82,"(1494.8627601239193, ({0, 1, 4, 5, 7, 10, 15, ...",1,12,"{160, 1, 163, 100, 133, 36, 26, 10, 12, 19, 11...",...,False,3767,0,8.362277,0.549560,1,3,True,True,True
240,4169,0.011931,18.323694,159.472844,8,82,"(1484.4315153492448, ({0, 1, 3, 4, 5, 8, 9, 12...",1,5,"{130, 132, 133, 100, 137, 74, 42, 11, 141, 143...",...,False,2133,0,7.042686,0.499686,1,3,True,True,True
241,4170,0.013469,20.179314,180.026202,4,82,"(1639.2320994076063, ({1, 2, 4, 5, 6, 7, 8, 9,...",1,6,NaN,...,False,102687,0,18.747141,0.664289,1,3,True,True,True


In [ ]:
df['Approximation and Heuristics Max Independent Sets'] = len(df['Approximation and Heuristics Max Independent Sets'])
df['Asteroidal Triple']= len(df['Asteroidal Triple'])
df['Approximation and Heuristics Max Cliques'] = len(df['Approximation and Heuristics Max Cliques'])
df['Approximation and Heuristics TSP'] = len(df['Approximation and Heuristics TSP'])
df['Approximation and Heuristics Diameter'] = len(df['Approximation and Heuristics Diameter'])
df['Approximation and Heuristics Min Wt Dominating Set'] = len(df['Approximation and Heuristics Min Wt Dominating Set'])
df['Approximation and Heuristics Randomized Partitioning'] = df['Approximation and Heuristics Randomized Partitioning'].apply(lambda k: eval(k)).apply(lambda k: k[0])
df['Approximation and Heuristics One Exchange'] = df['Approximation and Heuristics One Exchange'].apply(lambda k: eval(k)).apply(lambda k: k[0])

In [ ]:
df['1 Edge Connected'] = df['1 Edge Connected'].map({True:1 ,False:0})
df['2 Edge Connected'] = df['2 Edge Connected'].map({True:1 ,False:0})
df['3 Edge Connected'] = df['3 Edge Connected'].map({True:1 ,False:0})
df['Bridges'] = df['Bridges'].map({True:1 ,False:0})
df['Chordal'] = df['Chordal'].map({True:1 ,False:0})

In [ ]:
df['Randomized Partitioning Heuristic'] = df['Randomized Partitioning Heuristic'].apply(lambda k: eval(k))
df['Randomized Partitioning Heuristic'] = df['Randomized Partitioning Heuristic'].apply(lambda k: k[0])

In [ ]:
non_unique=list()
for column in df:
    if(len(pd.unique(df[column]))==1):
        non_unique.append(column)

non_unique

['Max weight matching',
 'Approximation and Heuristics Max Independent Sets',
 'Approximation and Heuristics Max Cliques',
 'Approximation and Heuristics Diameter',
 'Approximation and Heuristics Min Wt Dominating Set',
 'Approximation and Heuristics TSP',
 'Asteroidal Triple',
 'Chordal',
 'Num of isolates',
 'Connected Components',
 '1 Edge Connected']

In [ ]:
df=df.drop(non_unique, axis=1)

In [ ]:
df = df.fillna(0)

In [ ]:
df

,Subject,Average shortest path,Stoer Wagner cuts,Wiener Index,Dijkstra path,Randomized Partitioning Heuristic,Schizophrenic,Approximation and Heuristics Node Connectivity,Approximation and Heuristics Large Clique Size,Approximation and Heuristics Average Clustering,...,Graph colouring,Avg Degree,Global Clustering,Edges,Maximal Cliques,Non Randomness wrt Random Model,Transitivity,Diameter,2 Edge Connected,3 Edge Connected
0,2001,0.008042,19.841134,107.483595,8,1340.094230,0,10,17,0.465,...,19,17.512195,0.492320,2872,1813,-0.570663,0.503559,3,1,1
1,2002,0.012136,19.315941,162.212200,4,1491.632893,0,11,25,0.563,...,31,24.396341,0.541610,4001,5810,10.101452,0.548962,3,1,1
2,2003,0.010182,16.432399,136.090766,3,1302.932983,0,1,17,0.489,...,22,20.628049,0.499233,3383,3860,7.200119,0.492561,4,0,0
3,2004,0.009460,20.374916,126.446053,6,1283.965974,0,10,16,0.461,...,19,19.115854,0.462352,3135,3505,7.945752,0.464850,3,1,1
4,2005,0.011182,20.140523,149.462698,7,1304.706586,0,15,14,0.490,...,19,19.762195,0.477082,3241,3339,5.800827,0.472348,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,4156,0.010345,21.208409,138.275598,4,1426.881840,1,11,20,0.500,...,23,20.682927,0.497645,3392,4429,-4.078784,0.498773,3,1,1
239,4159,0.010946,21.128360,146.306992,9,1494.862760,1,12,24,0.531,...,30,23.012195,0.551732,3774,3767,8.362277,0.549560,3,1,1
240,4169,0.011931,18.323694,159.472844,8,1484.431515,1,5,21,0.526,...,24,18.987805,0.485025,3114,2133,7.042686,0.499686,3,1,1
241,4170,0.013469,20.179314,180.026202,4,1639.232099,1,6,35,0.623,...,41,34.280488,0.632228,5622,102687,18.747141,0.664289,3,1,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243 entries, 0 to 242
Data columns (total 28 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   Subject                                               243 non-null    int64  
 1   Average shortest path                                 243 non-null    float64
 2   Stoer Wagner cuts                                     243 non-null    float64
 3   Wiener Index                                          243 non-null    float64
 4   Dijkstra path                                         243 non-null    int64  
 5   Randomized Partitioning Heuristic                     243 non-null    float64
 6   Schizophrenic                                         243 non-null    int64  
 7   Approximation and Heuristics Node Connectivity        243 non-null    int64  
 8   Approximation and Heuristics Large Clique Size        243 no

In [ ]:
dfs = [df]

# TEST TRAIN SPLIT

In [ ]:
def preprocessing_pipeline(X):
  return X

In [ ]:
def split_fn(df):
  X = df.drop(['Schizophrenic','Subject'], axis=1)
  y = df['Schizophrenic']

  return X, y

In [ ]:
def main(df, clf, param):
  # data
  X, y = split_fn(df)
  X = preprocessing_pipeline(X)

  # grid
  gscv = GridSearchCV(clf, param, cv=10, scoring='accuracy' )
  gscv.fit(X, y)

  print(gscv.get_params())

  return gscv

#ML Models 

## LR

In [ ]:
clf = LogisticRegression()

In [ ]:
dt = clf
X, y = split_fn(df)
acc = []
f1 = []
recall = []
prec = []
spec = []
for r in range(1,11):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = r)
  dt.fit(X_train, y_train)
  pred = dt.predict(X_test)
  acc.append(accuracy_score(y_test, pred))
  f1.append(f1_score(y_test, pred))
  recall.append(recall_score(y_test, pred))
  prec.append(precision_score(y_test, pred))
  tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
  specificity = tn / (tn+fp)
  spec.append(specificity)
  
print('Average accccuracy: %.3f +/- %.3f' %(np.mean(acc), np.std(acc)))
print('Average specificity: %.3f +/- %.3f' %(np.mean(spec), np.std(spec)))
print('Average recall: %.3f +/- %.3f' %(np.mean(recall), np.std(recall)))
print('Average precision: %.3f +/- %.3f' %(np.mean(prec), np.std(prec)))
print('Average f1: %.3f +/- %.3f' %(np.mean(f1), np.std(f1)))

Average accccuracy: 0.612 +/- 0.069
Average specificity: 0.667 +/- 0.118
Average recall: 0.564 +/- 0.076
Average precision: 0.606 +/- 0.141
Average f1: 0.577 +/- 0.086


## SVM

In [ ]:
clf = SVC()
parameters = {'C': [0.1, 1, 10, 100,1000], 
              'gamma': [1, 0.1, 0.01, 0.001,0.0001],
              'kernel': ['rbf']} 

In [ ]:
res = []
for df in dfs: 
  res.append(main(df, clf, parameters))

{'cv': 10, 'error_score': nan, 'estimator__C': 1.0, 'estimator__break_ties': False, 'estimator__cache_size': 200, 'estimator__class_weight': None, 'estimator__coef0': 0.0, 'estimator__decision_function_shape': 'ovr', 'estimator__degree': 3, 'estimator__gamma': 'scale', 'estimator__kernel': 'rbf', 'estimator__max_iter': -1, 'estimator__probability': False, 'estimator__random_state': None, 'estimator__shrinking': True, 'estimator__tol': 0.001, 'estimator__verbose': False, 'estimator': SVC(), 'n_jobs': None, 'param_grid': {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['rbf']}, 'pre_dispatch': '2*n_jobs', 'refit': True, 'return_train_score': False, 'scoring': 'accuracy', 'verbose': 0}


In [ ]:
for clf in res:
  print(clf.best_estimator_, clf.best_params_, clf.best_score_)

SVC(C=1, gamma=0.0001) {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'} 0.8053333333333332


In [ ]:
dt = clf.best_estimator_

In [ ]:
X, y = split_fn(df)
acc = []
f1 = []
recall = []
prec = []
spec = []
for r in range(1,11):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = r)
  dt.fit(X_train, y_train)
  pred = dt.predict(X_test)
  acc.append(accuracy_score(y_test, pred))
  f1.append(f1_score(y_test, pred))
  recall.append(recall_score(y_test, pred))
  prec.append(precision_score(y_test, pred))
  tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
  specificity = tn / (tn+fp)
  spec.append(specificity)
  
print('Average accccuracy: %.3f +/- %.3f' %(np.mean(acc), np.std(acc)))
print('Average specificity: %.3f +/- %.3f' %(np.mean(spec), np.std(spec)))
print('Average recall: %.3f +/- %.3f' %(np.mean(recall), np.std(recall)))
print('Average precision: %.3f +/- %.3f' %(np.mean(prec), np.std(prec)))
print('Average f1: %.3f +/- %.3f' %(np.mean(f1), np.std(f1)))

Average accccuracy: 0.790 +/- 0.051
Average specificity: 0.900 +/- 0.052
Average recall: 0.676 +/- 0.105
Average precision: 0.852 +/- 0.076
Average f1: 0.747 +/- 0.074


## KNN

In [ ]:
clf = KNeighborsClassifier()
parameters = {'n_neighbors': list(range(3,50)),
            #   'weights': ['uniform','distance', None],
              'p': [1,2]
              }

In [ ]:
res = []
for df in dfs: 
  res.append(main(df, clf, parameters))

{'cv': 10, 'error_score': nan, 'estimator__algorithm': 'auto', 'estimator__leaf_size': 30, 'estimator__metric': 'minkowski', 'estimator__metric_params': None, 'estimator__n_jobs': None, 'estimator__n_neighbors': 5, 'estimator__p': 2, 'estimator__weights': 'uniform', 'estimator': KNeighborsClassifier(), 'n_jobs': None, 'param_grid': {'n_neighbors': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], 'p': [1, 2]}, 'pre_dispatch': '2*n_jobs', 'refit': True, 'return_train_score': False, 'scoring': 'accuracy', 'verbose': 0}


In [ ]:
for clf in res:
  print(clf.best_estimator_, clf.best_params_, clf.best_score_)

KNeighborsClassifier(n_neighbors=3, p=1) {'n_neighbors': 3, 'p': 1} 0.7241666666666666


### 10 fold cv

In [ ]:
dt = clf.best_estimator_

In [ ]:
X, y = split_fn(df)
acc = []
f1 = []
recall = []
prec = []
spec = []
for r in range(1,11):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = r)
  dt.fit(X_train, y_train)
  pred = dt.predict(X_test)
  acc.append(accuracy_score(y_test, pred))
  f1.append(f1_score(y_test, pred))
  recall.append(recall_score(y_test, pred))
  prec.append(precision_score(y_test, pred))
  tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
  specificity = tn / (tn+fp)
  spec.append(specificity)
  
print('Average accccuracy: %.3f +/- %.3f' %(np.mean(acc), np.std(acc)))
print('Average specificity: %.3f +/- %.3f' %(np.mean(spec), np.std(spec)))
print('Average recall: %.3f +/- %.3f' %(np.mean(recall), np.std(recall)))
print('Average precision: %.3f +/- %.3f' %(np.mean(prec), np.std(prec)))
print('Average f1: %.3f +/- %.3f' %(np.mean(f1), np.std(f1)))

Average accccuracy: 0.682 +/- 0.086
Average specificity: 0.640 +/- 0.117
Average recall: 0.732 +/- 0.089
Average precision: 0.647 +/- 0.113
Average f1: 0.683 +/- 0.090


## Decision Tree

In [ ]:
dt = DecisionTreeClassifier()
parameters = {'criterion': ['gini', 'entropy'],
              'max_depth': list(range(1,50,2)),
              }

In [ ]:
res = []
for df in dfs: 
  res.append(main(df, dt, parameters))

{'cv': 10, 'error_score': nan, 'estimator__ccp_alpha': 0.0, 'estimator__class_weight': None, 'estimator__criterion': 'gini', 'estimator__max_depth': None, 'estimator__max_features': None, 'estimator__max_leaf_nodes': None, 'estimator__min_impurity_decrease': 0.0, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 2, 'estimator__min_weight_fraction_leaf': 0.0, 'estimator__random_state': None, 'estimator__splitter': 'best', 'estimator': DecisionTreeClassifier(), 'n_jobs': None, 'param_grid': {'criterion': ['gini', 'entropy'], 'max_depth': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49]}, 'pre_dispatch': '2*n_jobs', 'refit': True, 'return_train_score': False, 'scoring': 'accuracy', 'verbose': 0}


In [ ]:
for clf in res:
  print(clf.best_estimator_, clf.best_params_, clf.best_score_)

DecisionTreeClassifier(max_depth=15) {'criterion': 'gini', 'max_depth': 15} 0.8276666666666668


### 10 fold CV

In [ ]:
dt = clf.best_estimator_

In [ ]:
X, y = split_fn(df)
acc = []
f1 = []
recall = []
prec = []
spec = []
for r in range(21,31):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = r)
  dt.fit(X_train, y_train)
  pred = dt.predict(X_test)
  acc.append(accuracy_score(y_test, pred))
  f1.append(f1_score(y_test, pred))
  recall.append(recall_score(y_test, pred))
  prec.append(precision_score(y_test, pred))
  tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
  specificity = tn / (tn+fp)
  spec.append(specificity)
  
print('Average accccuracy: %.3f +/- %.3f' %(np.mean(acc), np.std(acc)))
print('Average specificity: %.3f +/- %.3f' %(np.mean(spec), np.std(spec)))
print('Average recall: %.3f +/- %.3f' %(np.mean(recall), np.std(recall)))
print('Average precision: %.3f +/- %.3f' %(np.mean(prec), np.std(prec)))
print('Average f1: %.3f +/- %.3f' %(np.mean(f1), np.std(f1)))

Average accccuracy: 0.765 +/- 0.053
Average specificity: 0.701 +/- 0.071
Average recall: 0.828 +/- 0.052
Average precision: 0.734 +/- 0.079
Average f1: 0.776 +/- 0.060


## AdaBoost

In [ ]:
clf = AdaBoostClassifier()
parameters = {'n_estimators': list(range(3,101,10)),
              'learning_rate':[0.1,0.3,0.5,0.7,1.0],
              }

In [ ]:
res = []
for df in dfs: 
  res.append(main(df, clf, parameters))

{'cv': 10, 'error_score': nan, 'estimator__algorithm': 'SAMME.R', 'estimator__base_estimator': None, 'estimator__learning_rate': 1.0, 'estimator__n_estimators': 50, 'estimator__random_state': None, 'estimator': AdaBoostClassifier(), 'n_jobs': None, 'param_grid': {'n_estimators': [3, 13, 23, 33, 43, 53, 63, 73, 83, 93], 'learning_rate': [0.1, 0.3, 0.5, 0.7, 1.0]}, 'pre_dispatch': '2*n_jobs', 'refit': True, 'return_train_score': False, 'scoring': 'accuracy', 'verbose': 0}


In [ ]:
for clf in res:
  print(clf.best_estimator_, clf.best_params_, clf.best_score_)

AdaBoostClassifier(n_estimators=93) {'learning_rate': 1.0, 'n_estimators': 93} 0.7783333333333333


### 10 fold CV

In [ ]:
dt = clf.best_estimator_

In [ ]:
X, y = split_fn(df)
acc = []
f1 = []
recall = []
prec = []
spec = []
for r in list(range(1,11)):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = r)
  clf.fit(X_train, y_train)
  pred = clf.predict(X_test)
  acc.append(accuracy_score(y_test, pred))
  f1.append(f1_score(y_test, pred))
  recall.append(recall_score(y_test, pred))
  prec.append(precision_score(y_test, pred))
  tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
  specificity = tn / (tn+fp)
  spec.append(specificity)
print('Average accccuracy: %.3f +/- %.3f' %(np.mean(acc), np.std(acc)))
print('Average specificity: %.3f +/- %.3f' %(np.mean(spec), np.std(spec)))
print('Average recall: %.3f +/- %.3f' %(np.mean(recall), np.std(recall)))
print('Average precision: %.3f +/- %.3f' %(np.mean(prec), np.std(prec)))
print('Average f1: %.3f +/- %.3f' %(np.mean(f1), np.std(f1)))

Average accccuracy: 0.755 +/- 0.062
Average specificity: 0.721 +/- 0.109
Average recall: 0.807 +/- 0.086
Average precision: 0.721 +/- 0.103
Average f1: 0.755 +/- 0.061
